In [1]:

import os
import sys

# using from local copy
current_dir = os.getcwd()
sys.path.append(os.path.dirname(current_dir))

# Azure Image Analysis

In [13]:
from libs.community.langchain_community.tools.azure_ai_services import AzureAiServicesImageAnalysisTool

tool = AzureAiServicesImageAnalysisTool(
    azure_ai_services_key=os.environ["AZURE_AI_SERVICES_KEY"],
    azure_ai_services_endpoint=os.environ["AZURE_AI_SERVICES_ENDPOINT"],
)

In [14]:
dog = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwyXeKDN29AmZgZPLS7n0Bepe8QmVappBwZCeA3XWEbWNdiDFB"
warning_sign = "https://www.vanguardgroup.co.nz/wp-content/uploads/2021/12/4203-Toxic-Hazard.jpg"
people = "https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/images/family_photo.png"
text_read = "https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/images/gray-shirt-logo.jpg"

In [15]:
tool._run(warning_sign)

'Caption: a yellow warning sign with black text\nTags: text, font, yellow, logo, number, graphics, sign\nText: WARNING, TOXIC, HAZARD, vanguard'

In [9]:
tool._run(dog)

'Caption: a small white dog with a blue harness\nObjects: dog\nTags: animal, mammal, pet, outdoor, dog breed, white, dog, snout, chihuahua, puppy, standing'

In [11]:
tool._run(people)

'Caption: a group of people posing for a photo\nObjects: person, person, person, person\nTags: human face, clothing, person, smile, wall, happy, friendship, adult, laugh, adolescent, indoor, woman, people, fabric'

In [14]:
tool._run(text_read)

'Caption: a black sweatshirt with white text\nTags: clothing, top, person, active shirt, text, hoodie, jacket, sweatshirt, outerwear, indoor, sleeve\nText: Microsoft'

# Use with Agent

### 1. Prepare Toolkit

In [41]:
# https://python.langchain.com/v0.1/docs/integrations/toolkits/azure_ai_services/

# from libs.community.langchain_community.tools.azure_ai_services import AzureAiServicesImageAnalysisTool

# tool = AzureAiServicesImageAnalysisTool(
#     azure_ai_services_key=os.environ["AZURE_AI_SERVICES_KEY"],
#     azure_ai_services_endpoint=os.environ["AZURE_AI_SERVICES_ENDPOINT"],
# )

# tools = [tool]

from libs.community.langchain_community.agent_toolkits import AzureAiServicesToolkit

toolkit = AzureAiServicesToolkit(
    azure_ai_services_key=os.environ["AZURE_AI_SERVICES_KEY"],
    azure_ai_services_endpoint=os.environ["AZURE_AI_SERVICES_ENDPOINT"],
)

tools = toolkit.get_tools()
[tool.name for tool in tools]

['azure_ai_services_document_intelligence',
 'azure_ai_services_image_analysis',
 'azure_ai_services_speech_to_text',
 'azure_ai_services_text_to_speech',
 'azure_ai_services_text_analytics_for_health']

### 2. Prepare Prompt

In [39]:
from libs.langchain.langchain import hub

hub_api_key = os.environ["LANGCHAIN_HUB_API_KEY"]
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=hub_api_key)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'structured-chat-agent', 'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nVali

### 3. Prepare Chat Model and Agent

In [35]:
from libs.partners.openai.langchain_openai import AzureChatOpenAI
from libs.langchain.langchain.agents import AgentExecutor, create_structured_chat_agent

chat = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment="gpt-4o"
)

agent = create_structured_chat_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

### 4. Execute the Agent

In [36]:
# this works
agent_executor.invoke(
    {
        "input": "What can I make with these ingredients? " +
        "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
    }
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "azure_ai_services_image_analysis",
  "action_input": {
    "query": "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
  }
}
```Caption: a group of eggs and flour in bowls
Objects: Egg, Egg, Food
Tags: dairy, ingredient, indoor, thickening agent, food, mixing bowl, powder, flour, egg, bowlAction:
```
{
  "action": "Final Answer",
  "action_input": "With eggs and flour, you can make a variety of recipes including pancakes, crepes, pasta, or even a simple cake. The specific recipe would depend on the quantities and any additional ingredients you might have."
}
```

> Finished chain.


{'input': 'What can I make with these ingredients? https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png',
 'output': 'With eggs and flour, you can make a variety of recipes including pancakes, crepes, pasta, or even a simple cake. The specific recipe would depend on the quantities and any additional ingredients you might have.'}

In [44]:
# example from https://python.langchain.com/v0.2/docs/integrations/tools/azure_ai_services/#use-within-an-agent
# maybe indentation problem etc?
agent_executor.invoke(
    {
        "input": "What can I make with these ingredients? "
        + "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
    }
)



> Entering new AgentExecutor chain...
{
  "action": "azure_ai_services_image_analysis",
  "action_input": {
    "query": "Identify ingredients from the image"
  }
}

RuntimeError: Error while running AzureAiImageAnalysisTool: Invalid image path: Identify ingredients from the image